<a href="https://colab.research.google.com/github/landges/automatic-octo-machine/blob/master/analize_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!pip install tokenization
!pip install nltk
!pip install pymorphy2

In [52]:
from google.colab import drive
import re
import tokenization
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
import pymorphy2
import matplotlib.pyplot as plt
import os
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import random
import math
import prettytable
drive.mount('/content/drive')
nltk.download('stopwords')

stop_words = list(set(stopwords.words("russian")))

def normalize_tokens(tokens):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(tok)[0].normal_form for tok in tokens]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Считывание данных

In [0]:
positive_text=pd.read_csv("/content/drive/My Drive/Colab Notebooks/тексты/positive.csv",delimiter=';',
                          names=['id','tdate','tmane','ttext','ttype','trep','trtw','tfav','tstcount','tfol','tfrien','listcount'])
texts_pos=[positive_text.ttext[t] for t in range(len(positive_text.ttext))]

negative_text=pd.read_csv("/content/drive/My Drive/Colab Notebooks/тексты/negative.csv",delimiter=';',
                          names=['id','tdate','tmane','ttext','ttype','trep','trtw','tfav','tstcount','tfol','tfrien','listcount'])
texts_neg=[negative_text.ttext[t] for t in range(len(negative_text.ttext))]
text=''.join(texts_pos+texts_neg)

# Анализ коллекций

In [54]:
prog=re.compile('[А-Яа-я]+')
t1=prog.findall(text.lower())
morph = pymorphy2.MorphAnalyzer()
t2=[morph.parse(tok)[0].normal_form for tok in t1 if tok not in stop_words]
t3=nltk.FreqDist(t2)
print(t3.most_common(20))

[('это', 20796), ('хотеть', 11641), ('день', 9974), ('такой', 8882), ('сегодня', 8570), ('очень', 7612), ('быть', 7297), ('год', 7249), ('ты', 7059), ('один', 6879), ('просто', 6790), ('мой', 6404), ('мочь', 6292), ('человек', 6025), ('знать', 5940), ('весь', 5803), ('любить', 5592), ('завтра', 5254), ('вс', 4869), ('вообще', 4851)]


In [55]:
bi_grams=list(nltk.bigrams(t1))
fr_dg=nltk.FreqDist(bi_grams)
print(fr_dg.most_common(10))

[(('у', 'меня'), 10014), (('я', 'не'), 5515), (('а', 'я'), 4855), (('и', 'не'), 3453), (('не', 'могу'), 3179), (('что', 'я'), 3041), (('у', 'нас'), 2826), (('что', 'то'), 2595), (('не', 'знаю'), 2301), (('как', 'же'), 2232)]


In [56]:
def sent_token(text):
    text=text.lower()
    words=re.findall(r"[\w']+|[.,!?;:{}() \n]",text)
    words=[word for word in words if word not in r'[ .,;:?!@#$%^&*()\n]']
    return words

n=5
dic={}
for s in texts_pos+texts_neg:
    s=sent_token(s)
    for wi in range(len(s)):
       ind=min(10,len(s)-wi)
       for wi2 in range(1,ind):
           dic[(s[wi], s[wi+wi2])]=[]

for s in texts_pos+texts_neg:
    s=sent_token(s)
    for wi in range(len(s)):
       ind=min(10,len(s)-wi)
       for wi2 in range(1,ind):
           dic[(s[wi], s[wi+wi2])].append(wi2-1)

l_dic=list(dic.items())
l_d=[]
def sor_s(input):
    return input[1]
def sor_count(input):
    return input[2]
for item in l_dic:
    if len(item[1])<4:
        continue
    item=list(item)
    d=sum(item[1])/len(item[1])
    r=random.randint(0,len(item[1])-1)
    s=math.sqrt(sum([(t-item[1][r])**2 for t in item[1]])/(len(item[1])))
    if 0.3<=s<=0.6:
      item1=[d,s,len(item[1]),item[0][0],item[0][1]]
      l_d.append(item1)
    else: continue
l_d.sort(key=sor_s)
l_d.sort(key=sor_count,reverse=True)
table = PrettyTable(['d','s','count','word1','word2'])
for row in l_d:
  table.add_row(row)
print(table)

+----------------------+---------------------+-------+-----------------------------------+----------------------+
|          d           |          s          | count |               word1               |        word2         |
+----------------------+---------------------+-------+-----------------------------------+----------------------+
| 0.06840488052265743  |  0.5864076989940155 | 33521 |                 t                 |          co          |
| 0.06353075719272902  |  0.5626993447179812 | 33228 |                http               |          t           |
|  1.0553637409897632  |  0.5068433496785097 | 33018 |                http               |          co          |
| 0.03202846975088968  | 0.42602190310089477 |  1124 |                 со                |         мной         |
|  1.026813880126183   |  0.3661548861218635 |  634  |                 до                |         пор          |
| 0.03403565640194489  | 0.46428348798155383 |  617  |                 до               

In [57]:
from nltk.collocations import *
n=100
text=''.join(texts_pos+texts_neg)
w=sent_token(text)
bm=nltk.collocations.BigramAssocMeasures()
f= BigramCollocationFinder.from_words(w)
f.apply_freq_filter(5)

raw_freq_rank=[' '.join(i) for i in f.nbest(bm.raw_freq,n)]
print(raw_freq_rank)

tscore_rank=[' '.join(i) for i in f.nbest(bm.student_t,n)]
print(tscore_rank)

pmi=[' '.join(i) for i in f.nbest(bm.pmi,n)]
print(pmi)

chi2=[' '.join(i) for i in f.nbest(bm.chi_sq,n)]
print(chi2)

llr=[' '.join(i) for i in f.nbest(bm.likelihood_ratio,n)]
print(llr)


['t co', 'http t', 'у меня', 'я не', 'а я', 'и не', 'не могу', '\n я', 'что я', 'у нас', 'что то', 'не знаю', 'd \n', '\n а', 'как же', 'd http', 'а у', 'у тебя', 'и я', 'новый год', 'не хочу', 'потому что', 'из за', 'но я', 'а то', 'ничего не', 'как то', 'то что', 'я так', 'я в', '\n \n', 'я уже', 'я тоже', 'так и', 'а ты', '\n и', 'даже не', 'в школу', 'как я', 'это не', 'что не', 'а мне', 'что у', 'со мной', 'никто не', 'все равно', 'что ты', 'кто то', 'я хочу', 'с тобой', '\n но', 'а потом', 'никогда не', 'не было', 'но не', 'и все', 'мне не', 'и в', '\n ну', 'так что', 'еще и', 'то я', 'в этом', 'меня не', 'не будет', 'я бы', 'же я', 'что это', 'я тебя', 'а не', 'то не', 'ты не', 'ну вот', 'не в', 'вот и', 'я и', 'что за', 'так не', 'ну и', 'на улице', 'да я', 'я буду', 'уже не', '\n не', 'а вот', 'что в', 'с кем', 'какой то', 'вообще не', 'в школе', 'не так', 'не надо', 'жаль что', 'я просто', 'еще не', 'а в', 'я люблю', 'с \n', 'доброе утро', 'знаю что']
['t co', 'http t', 'у ме

In [58]:
from gensim.summarization import keywords
text=''.join(texts_pos[:250])
w=sent_token(text)
norma=normalize_tokens(w)
normal_text=' '.join(norma)

kw=keywords(normal_text)
print(kw)

весь
http
https
мои
так
быть много
как
сегодня
такои
еще
вот что значит
уже
это конечно
очень
для
они
есть
тут
просто один
смотреть там
кто
ведь она
только
свои
когда
наш
надо
два день
спасибо
любить или нет
какои
хорошии drt
раз
мочь
большои
после
тоже
тот
тогда
теперь
можно
время
знать
этот
мама
неделя
сам
опять
рада
работать клоун
понять
без
под
прекрасныи
хорошо
если
самыи
про
год
рождение
написать
человек
давать
даже настроение
при
ночь
пора
снег
получаться
ждать
утром
наконец
интересныи
вообще
идти завтра
заити
наверное
минута
школа
читать
говорят
похожии
ааа
равно
говорить
чем
хотеть
круглыи
ахуенно
девушка
утро проснуться
сторона
хотеться писать
вечером
красота
всего
новыи тема
невозможность
likhodedovamary
патихардит
спиздил
one_enclave
_staacyyy
широко
совсем
случаи
voryi_ybivai
мишка
маленькии
сказать
километр число
слово
будущии
влюбляться
наити
спать
новогоднии
фильм
посмотреть
часы
где
зима
сделать
взять
выпить
проити программа
добрыи
ваш
всегда
тысяча
вчера приехать
дост

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('averaged_perceptron_tagger')
tfidf=TfidfVectorizer()
def get_feature_name_by_tfidf(text_to_process):
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4), min_df=0, stop_words=stop_words)
    tfidf_matrix = tf.fit_transform(text_to_process.split(','))
    tagged = nltk.pos_tag(tf.get_feature_names())
    feature_names_with_tags = {k: v for k, v in dict(tagged).items() if v != 'VBP'}
    return list(feature_names_with_tags.keys())

fil=open("/content/drive/My Drive/Colab Notebooks/тексты/35000Толстой Лев. Война и мир. Том 3 - royallib.ru.txt",'r')
text=fil.read()
keyword_s=get_feature_name_by_tfidf(text)
print(keyword_s[-100:])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
['ясное вечера небо', 'ясное вечера небо застлано', 'ясной', 'ясной мысли', 'ясной мысли смерти', 'ясном', 'ясном утреннем', 'ясном утреннем свете', 'ясном утреннем свете смотрел', 'ясностью', 'ясностью глубиною', 'ясностью действительности', 'ясностью действительности послышались', 'ясностью действительности послышались бум', 'ясностью пришла', 'ясностью пришла голову', 'ясностью пришла голову пыталась', 'ясностью силой', 'ясностью такими', 'ясностью такими подробностями', 'ясною', 'ясною весьма', 'ясною весьма понравилась', 'ясною весьма понравилась roi', 'ясную', 'ясную характеристику', 'ясную характеристику человека', 'ясную характеристику человека пфуль', 'ясные', 'ясные минуты', 'ясные минуты знал', 'ясные страшные', 'ясные страшные мысли', 'ясные страшные мысли оставляли', 'ясный', 'ясный авгус